### Time Series: Explore Lesson
**Corey Solitaire**   
10.28.2020

In [1]:
# data manipulation 
import numpy as np
import pandas as pd

from datetime import datetime
import itertools

# data visualization 
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm

%matplotlib inline

from sklearn.model_selection import TimeSeriesSplit

# ignore warnings
import warnings
warnings.filterwarnings("ignore")

from acquire import get_store_data, opsd_germany_daily
from prepare import prepped_store_df, prepped_energy_df

# Store DF

In [ ]:
df = get_store_data()
df = prepped_store_df(df)
df.head()

# Energy DF

In [ ]:
df = opsd_germany_daily()
df = prepped_energy_df(df)
df.head()

### Using your store items data you prepped in lesson 2 exercises:

1. Split your data into train and test using the sklearn.model_selection.TimeSeriesSplit method.

2. Validate your splits by plotting X_train and y_train.

3. Plot the weekly average & the 7-day moving average. Compare the 2 plots.

4. Plot the daily difference. Observe whether usage seems to vary drastically from day to day or has more of a smooth transition.

5. Plot a time series decomposition.

6. Create a lag plot (day over day).

7. Run a lag correlation.

### Using your OPS data you prepped in lesson 2 exercises:

1. Split your data into train and test using the percent cutoff method.

2. Validate your splits by plotting X_train and y_train.

3. Plot the weekly average & the 7-day moving average. Compare the 2 plots.

4. Group the electricity consumption time series by month of year, to explore annual seasonality.

5. Plot the daily difference. Observe whether usage seems to vary drastically from day to day or has more of a smooth transition.

6. Plot a time series decomposition. Takeaways?

### Bonus:

If time:

For each store I want to see how many items were sold over a period of time, for each item. Find a way to chart this. Hints: Subplots for the piece with the fewest distinct values (like store), x = time, y = count, color = item. If you have too many distinct items, you may need to plot the top n, while aggregating the others into an 'other' bucket. 